In [2]:
# Mount drive 
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "//content//gdrive//My Drive//My DeepLearning//"
import os
os.path.isdir(path)   

True

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras import Model, layers
from keras.models import load_model, model_from_json
import tensorflow as tf
import PIL

In [ ]:
#%cd /content/gdrive/'My Drive'/'My DeepLearning'/'Practice'
#!unzip 'alien-vs-predator-images.zip'

In [ ]:
# path for Kaggle kernels
input_path = path + "/Alien-vs-Predator-images/"
os.path.isdir(input_path) 

True

In [ ]:
train_datagen = ImageDataGenerator(
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    input_path + 'train',
    batch_size=32,
    class_mode='binary',
    target_size=(224,224))

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

validation_generator = validation_datagen.flow_from_directory(
    input_path + 'validation',
    shuffle=False,
    class_mode='binary',
    target_size=(224,224))

Found 694 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [ ]:
conv_base = ResNet50(
    include_top=False,
    weights='imagenet')

for layer in conv_base.layers:
    layer.trainable = False

In [ ]:
x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x) 
predictions = layers.Dense(4, activation='softmax')(x)
model = Model(conv_base.input, predictions)

In [ ]:
optimizer = keras.optimizers.Adam()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=347 // 32,  # added in Kaggle
                              epochs=3,
                              validation_data=validation_generator,
                              validation_steps=10  # added in Kaggle
                             )

Epoch 1/3
10/10 [==============================] - 57s 6s/step - loss: 0.1652 - accuracy: 0.9387 - val_loss: 0.4756 - val_accuracy: 0.8500
Epoch 2/3
10/10 [==============================] - 37s 4s/step - loss: 0.1490 - accuracy: 0.9344
Epoch 3/3
10/10 [==============================] - 34s 3s/step - loss: 0.1115 - accuracy: 0.9531


In [ ]:
# save  
model.save(path + 'Transfer Learning Models/Alien-vs-Predator-images.h5')

OSError: ignored

In [ ]:
# load
model = load_model(path + 'Transfer Learning Models/Object-Dataset-images.h5')

In [ ]:
import os
test_path = input_path + "/validation/cats/"
validation_img_paths = os.listdir(test_path)
len(validation_img_paths)

FileNotFoundError: ignored

In [ ]:
img_list = [Image.open(test_path + img_path) for img_path in validation_img_paths]

In [ ]:
validation_batch = np.stack([preprocess_input(np.array(img.resize((224,224))))
                             for img in img_list])

In [ ]:
pred_probs = model.predict(validation_batch)
pred_probs

array([[9.99597371e-01, 4.02621867e-04, 5.19766807e-08, 4.05770351e-08],
       [9.99970675e-01, 2.92777349e-05, 9.98223015e-09, 5.84639803e-09],
       [9.99998331e-01, 1.72608623e-06, 2.07180406e-09, 1.59435654e-09],
       [9.99931335e-01, 6.84539846e-05, 3.11005230e-08, 2.20819956e-07],
       [9.99957323e-01, 4.26582083e-05, 2.17425522e-09, 1.29526417e-10],
       [9.98211265e-01, 1.78640394e-03, 2.35914376e-06, 3.29538992e-08],
       [9.99732912e-01, 2.66949559e-04, 7.52452181e-08, 2.53056758e-08],
       [9.99886513e-01, 1.13495560e-04, 1.36319322e-09, 2.82439877e-10],
       [9.99978423e-01, 2.12350242e-05, 6.61911912e-08, 2.32427524e-07],
       [9.99973178e-01, 2.68660078e-05, 1.24816157e-08, 1.97355767e-08],
       [9.99963999e-01, 3.52622810e-05, 3.75891943e-07, 3.47712870e-07],
       [9.99969602e-01, 3.03032466e-05, 3.06388053e-08, 6.38174242e-08],
       [9.99989986e-01, 1.00099314e-05, 1.36310818e-09, 7.64964447e-09],
       [9.99931455e-01, 6.84051847e-05, 4.63590091e

In [ ]:
import numpy as np
ObjectList = ["cats", "dogs", "horses", "humans"]
index_max = np.argmax(pred_probs[0])
value_max = np.max(pred_probs[0])
print(ObjectList[index_max])
print(value_max)

cats
0.9995974


In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing import image

i = 0
for filename in validation_img_paths:
    img_path = test_path + filename
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    index_max = np.argmax(pred_probs[i])
    value_max = np.max(pred_probs[i])
    i = i + 1

    plt.imshow(img)
    plt.show()
    display(ObjectList[index_max] + " with probs " + str(value_max))

In [ ]:
import os
test_path = input_path + "/validation/humans/"
validation_img_paths = os.listdir(test_path)
display(len(validation_img_paths))
img_list = [Image.open(test_path + img_path) for img_path in validation_img_paths]
validation_batch = np.stack([preprocess_input(np.array(img.resize((224,224)))) for img in img_list])


30

In [ ]:
pred_probs = model.predict(validation_batch)
pred_probs

array([[5.99968298e-05, 5.84856789e-05, 4.14563436e-03, 9.95735884e-01],
       [5.81841823e-06, 9.29396265e-05, 1.70870102e-04, 9.99730408e-01],
       [5.42377757e-06, 1.45743479e-06, 4.97277069e-05, 9.99943376e-01],
       [1.98235466e-05, 3.80492493e-05, 3.66392014e-05, 9.99905467e-01],
       [2.53130303e-04, 5.07366727e-04, 1.60601630e-03, 9.97633457e-01],
       [4.16677467e-05, 1.69345731e-04, 2.66895932e-03, 9.97120023e-01],
       [1.73495704e-04, 9.08243121e-04, 1.24689052e-03, 9.97671306e-01],
       [5.08745279e-06, 3.66085696e-05, 1.08102868e-04, 9.99850154e-01],
       [4.66994788e-06, 2.11982679e-05, 8.68271527e-05, 9.99887347e-01],
       [2.25727345e-05, 5.58623287e-04, 9.86883938e-01, 1.25348847e-02],
       [1.91797371e-05, 2.73923826e-04, 5.78048965e-03, 9.93926466e-01],
       [3.86954162e-05, 2.04208645e-05, 4.19897970e-06, 9.99936700e-01],
       [8.61486420e-04, 4.42044530e-03, 2.63167620e-01, 7.31550455e-01],
       [1.00391353e-05, 5.35686922e-05, 1.61464289e

In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing import image

i = 0
for filename in validation_img_paths:
    img_path = test_path + filename
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    plt.imshow(img)
    plt.show()
    
    index_max = np.argmax(pred_probs[i])
    value_max = np.max(pred_probs[i])
    i = i + 1
    
    display(ObjectList[index_max] + " with probs " + str(value_max))

NameError: ignored